In [5]:
import trio

Q4: Given this code snippet, what would be the order of what is printed?

In [6]:
async def example(): 
    print("Start")
    await trio.sleep(1)
    print("Middle")
    await trio.sleep(1)
    print("End")

async def main(): 
    async with trio.open_nursery() as nursery: 
        nursery.start_soon(example)
        print("Main")

trio.run(main)

Main
Start
Middle
End


Trace the execution flow of this code and identify when each task runs and completes:

In [ ]:
async def task_with_subtasks():
    print("Parent: starting")
    async with trio.open_nursery() as subtask_nursery:
        subtask_nursery.start_soon(subtask, 1)
        print("Parent: spawned first subtask")
        await trio.sleep(0.5)
        subtask_nursery.start_soon(subtask, 2)
        print("Parent: spawned second subtask")
    print("Parent: subtasks complete")

async def subtask(number):
    print(f"Subtask {number}: starting")
    await trio.sleep(1)
    print(f"Subtask {number}: finished")

trio.run(task_with_subtasks)

What will this code output and why? Pay attention to the behavior of the Semaphore:

In [4]:
async def worker(id, semaphore):
    print(f"Worker {id} waiting for semaphore")
    async with semaphore:
        print(f"Worker {id} acquired semaphore")
        await trio.sleep(1)
        print(f"Worker {id} releasing semaphore")

async def main():
    semaphore = trio.Semaphore(2)
    
    async with trio.open_nursery() as nursery:
        for i in range(1, 5):
            nursery.start_soon(worker, i, semaphore)
            await trio.sleep(0.1)
            
    print("All workers completed")

trio.run(main)

Worker 1 waiting for semaphore
Worker 1 acquired semaphore
Worker 2 waiting for semaphore
Worker 2 acquired semaphore
Worker 3 waiting for semaphore
Worker 4 waiting for semaphore
Worker 1 releasing semaphore
Worker 3 acquired semaphore
Worker 2 releasing semaphore
Worker 4 acquired semaphore
Worker 3 releasing semaphore
Worker 4 releasing semaphore
All workers completed


What will be the output of this code, and why?

In [3]:
async def task(n):
    print(f"Task {n} starting")
    if n == 2:
        raise ValueError("Task 2 fails")
    await trio.sleep(0.5)
    print(f"Task {n} ending")

async def main():
    try:
        async with trio.open_nursery() as nursery:
            for i in range(1, 4):
                nursery.start_soon(task, i)
    except Exception as e:
        print(f"Caught: {e}")
    print("Main complete")

trio.run(main)

Task 1 starting
Task 2 starting
Task 3 starting
Caught: Exceptions from Trio nursery (1 sub-exception)
Main complete


Trace the execution of this code with race conditions. What are the possible outputs?

In [2]:
# shared_counter = 0

async def increment_counter(id, n_times):
    global shared_counter
    for _ in range(n_times):
        current = shared_counter
        await trio.sleep(0.01)  # Simulate some work
        shared_counter = current + 1
        print(f"Task {id} incremented to {shared_counter}")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(increment_counter, 1, 3)
        nursery.start_soon(increment_counter, 2, 3)
    
    print(f"Final counter value: {shared_counter}")

trio.run(main)

Task 1 incremented to 1
Task 2 incremented to 1
Task 2 incremented to 2
Task 1 incremented to 2
Task 2 incremented to 3
Task 1 incremented to 3
Final counter value: 3


In [ ]:
# Review the following code snippet and identify the bug -> does not execute async bc of the await
async def process_data(items):
    results = []
    async with trio.open_nursery() as nursery:
        for item in items:
            result = await process_item(item)
            results.append(result)
    return results

In [14]:
# Review the code and describe the bug: -> task never completes because of timeout function
async def run_with_timeout():
    with trio.move_on_after(1):
        await long_running_task()

async def long_running_task():
    await trio.sleep(2)
    print("Task completed")

trio.run(run_with_timeout)

In [ ]:
# What's the problem with this code? -> client is created each time and never closed
async def fetch_data_repeatedly():
    while True:
        client = httpx.AsyncClient()
        try:
            response = await client.get("https://api.example.com/status")
            if response.status_code == 200:
                print("Service is up!")
            else:
                print(f"Service returned: {response.status_code}")
        except Exception as e:
            print(f"Error occurred: {e}")
        
        await trio.sleep(60)

In [13]:
# what is wrong with this code? where is the await missing? 
async def fetch_data(user_id):
    await trio.sleep(1)
    return {'id': user_id, 'data': f'User data for {user_id}'}

async def process_users(user_ids):
    all_data = {}
    
    async def process_one_user(uid):
        data = fetch_data(uid) 
        all_data[uid] = data
    
    async with trio.open_nursery() as nursery:
        for uid in user_ids:
            nursery.start_soon(process_one_user, uid)
    
    return all_data

trio.run(process_users, [1, 2, 3])

{1: <coroutine object fetch_data at 0x10eca7e80>,
 2: <coroutine object fetch_data at 0x10eca7b80>,
 3: <coroutine object fetch_data at 0x10eca7c40>}

In [9]:
# what is wrong with this code? # nursery not properly open 

async def process_task(result_dict):
    await trio.sleep(2)
    result_dict['status'] = 'completed'

async def main():
    results = {}
    
    nursery = trio.open_nursery()
    nursery.start_soon(process_task, results)
    
    # Do some other work
    await trio.sleep(1)
    print(f"Results: {results}")  # Expect to see status='completed'

trio.run(main)

AttributeError: 'NurseryManager' object has no attribute 'start_soon'

### Implementation Question:

In [ ]:
# A 
import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

async def main():
    await say_hello()
    await say_world()

trio.run(main)

# B

import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(say_hello)
        nursery.start_soon(say_world)

trio.run(main)

# C
import trio

def say_hello():
    trio.sleep(1)
    print("Hello")

def say_world():
    trio.sleep(2)
    print("World")

async def main():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(say_hello)
        nursery.start_soon(say_world)

trio.run(main)

# D
import trio

async def say_hello():
    await trio.sleep(1)
    print("Hello")

async def say_world():
    await trio.sleep(2)
    print("World")

def main():
    nursery = trio.open_nursery()
    nursery.start_soon(say_hello)
    nursery.start_soon(say_world)

trio.run(main)